In [2]:
pip install opencv-python

     |████████████████████████████████| 43.7 MB 2.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
from skimage.exposure import rescale_intensity
from skimage.segmentation import slic
from skimage.util import img_as_float
from skimage import io
import numpy as np
import os

In [2]:
Folder_name="augmented_image"

In [16]:
def resize_image(image,w,h):
    image=cv2.resize(image,(w,h))
    #cv2.imwrite(Folder_name+"/Resize-"+str(w)+"*"+str(h)+Extension, image)
    return image
#crop
def crop_image(image,y1,y2,x1,x2):
    image=image[y1:y2,x1:x2]
    #cv2.imwrite(Folder_name+"/Crop-"+str(x1)+str(x2)+"*"+str(y1)+str(y2)+Extension, image)
    return image
def padding_image(image,topBorder,bottomBorder,leftBorder,rightBorder,color_of_border=[0,0,0]):
    image = cv2.copyMakeBorder(image,topBorder,bottomBorder,leftBorder,
        rightBorder,cv2.BORDER_CONSTANT,value=color_of_border)
    #cv2.imwrite(Folder_name + "/padd-" + str(topBorder) + str(bottomBorder) + "*" + str(leftBorder) + str(rightBorder) + Extension, image)
    return image
def flip_image(image,dir):
    image = cv2.flip(image, dir)
    #cv2.imwrite(Folder_name + "/flip-" + str(dir)+Extension, image)
    return image
def superpixel_image(image,segments):
    seg=segments

    def segment_colorfulness(image, mask):
        # split the image into its respective RGB components, then mask
        # each of the individual RGB channels so we can compute
        # statistics only for the masked region
        (B, G, R) = cv2.split(image.astype("float"))
        R = np.ma.masked_array(R, mask=mask)
        G = np.ma.masked_array(B, mask=mask)
        B = np.ma.masked_array(B, mask=mask)

        # compute rg = R - G
        rg = np.absolute(R - G)

        # compute yb = 0.5 * (R + G) - B
        yb = np.absolute(0.5 * (R + G) - B)

        # compute the mean and standard deviation of both `rg` and `yb`,
        # then combine them
        stdRoot = np.sqrt((rg.std() ** 2) + (yb.std() ** 2))
        meanRoot = np.sqrt((rg.mean() ** 2) + (yb.mean() ** 2))

        # derive the "colorfulness" metric and return it
        return stdRoot + (0.3 * meanRoot)

    orig = cv2.imread(image)
    vis = np.zeros(orig.shape[:2], dtype="float")

    # load the image and apply SLIC superpixel segmentation to it via
    # scikit-image
    image = io.imread(image)
    segments = slic(img_as_float(image), n_segments=segments,
                    slic_zero=True)
    for v in np.unique(segments):
        # construct a mask for the segment so we can compute image
        # statistics for *only* the masked region
        mask = np.ones(image.shape[:2])
        mask[segments == v] = 0

        # compute the superpixel colorfulness, then update the
        # visualization array
        C = segment_colorfulness(orig, mask)
        vis[segments == v] = C
    # scale the visualization image from an unrestricted floating point
    # to unsigned 8-bit integer array so we can use it with OpenCV and
    # display it to our screen
    vis = rescale_intensity(vis, out_range=(0, 255)).astype("uint8")

    # overlay the superpixel colorfulness visualization on the original
    # image
    alpha = 0.6
    overlay = np.dstack([vis] * 3)
    output = orig.copy()
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    # cv2.imshow("Visualization", vis)
    #cv2.imwrite(Folder_name + "/superpixels-" + str(seg) + Extension, output)
    return image
def invert_image(image,channel):
    # image=cv2.bitwise_not(image)
    image=(channel-image)
    #cv2.imwrite(Folder_name + "/invert-"+str(channel)+Extension, image)
    return image
def add_light(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")

    image=cv2.LUT(image, table)
    
    return image
def add_light_color(image, color, gamma=1.0):
    invGamma = 1.0 / gamma
    image = (color - image)
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")

    image=cv2.LUT(image, table)
    return image
def saturation_image(image,saturation):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    v = image[:, :, 2]
    v = np.where(v <= 255 - saturation, v + saturation, 255)
    image[:, :, 2] = v

    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
    return image 
def hue_image(image,saturation):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    v = image[:, :, 2]
    v = np.where(v <= 255 + saturation, v - saturation, 255)
    image[:, :, 2] = v

    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
    return image 

In [24]:
PATH = os.getcwd()
print(PATH)
data_path = PATH + '/new data'
data_dir_list = os.listdir(data_path)
p=os.path.join(PATH,"aug_data_6")
#os.mkdir(p)

img_rows = 128
img_cols = 128
num_channel = 1

img_data_list = []
i=0
for dataset in data_dir_list:
    if(dataset=='.ipynb_checkpoints'):
        continue
    
    image = cv2.imread(data_path + '/' + dataset)
    #print(image)
    #superpixel_image(image,100)
    #image=resize_image(image,70,70)
    #image=crop_image(image,10,10,0,10)
    image=padding_image(image,100,0,0,0)
    image=flip_image(image,0)
    image=invert_image(image,255)
    image=hue_image(image,15)
    image=add_light_color(image,255,1.5)
    image=saturation_image(image,60)
    cv2.imwrite(p+'/'+dataset,image)

/Users/arvt/pokedox


In [45]:
dataset

'184.jpg'

In [43]:
data_dir_list[600]

'.ipynb_checkpoints'

In [ ]:
data_dir_list

In [29]:
print(image)

None


'data_path/dataset'

In [6]:
import os 
